# AWS Docker tests

## AWS Authentication

In [26]:
import boto3
import sagemaker

sagemaker.__version__

'2.59.3'

In [27]:
def aws_account_info():
    account_id = boto3.client('sts').get_caller_identity().get('Account')
    region = boto3.Session().region_name
    role = sagemaker.get_execution_role()
    ecr_base= '{}.dkr.ecr.{}.amazonaws.com'.format(account_id, region) # , ecr_repository + tag)
    print('account=', account_id)
    print('region=', region)
    print('role=', role)
    print("ECR base url=", ecr_base)

In [28]:
aws_account_info()

account= 667175760002
region= eu-central-1
role= arn:aws:iam::667175760002:role/service-role/AmazonSageMaker-ExecutionRole-20211001T082927
ECR base url= 667175760002.dkr.ecr.eu-central-1.amazonaws.com


In [29]:
import subprocess
import sys

cmd = "/bin/netstat"
 
## run it ##
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)

out= p.communicate()

if False:
## But do not wait till netstat finish, start displaying output immediately ##
    while True:
        out = p.stdout.read(1)
        if out == '' and p.poll() != None:
            break
        if out != '':
            sys.stdout.write(out)
            sys.stdout.flush()

In [ ]:
print(out[0].decode('UTF-8'))

## Set up basic container 

In [4]:
'sage' in 'sagemaker-abc'

True

In [30]:
import tempfile
import subprocess
import os

def run_cmd(cmd, dryrun= True, check= True):
    print(cmd)
    if dryrun is not True:
        # print("exec")
        p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        out, err= p.communicate()
        if err is not None:
            err= err.decode('UTF-8')
        if out is not None:
            out= out.decode('UTF-8')
        p_status = p.wait()

        print(out)
        
        if p_status != 0:
            print("** Non-zero status returned! **")
            if err is not None:
                print(err)
            if check:
                assert p_status==0, "Command execution failed. Abort!"
        # subprocess.run(cmd, shell= True, check= check)
    
def make_Docker(dirname='docker_test1', libs= ['pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1'], 
                img_name= 'docker-auto-test', img_tag='latest', ecr_base= None, dryrun= False,
                cleanup= True):
    
    if 'sagemaker' not in img_name:
        img_name= 'sagemaker-'+img_name
        
    if dirname is None:
        tmpdirname= tempfile.TemporaryDirectory(prefix='docker_tmpdir_')  
        print('created temporary directory', tmpdirname.name)
        dirname= tmpdirname.name
    else:
        if os.path.isdir(dirname) is False:
            os.makedirs(dirname)
            
    docker_fname= dirname + "/Dockerfile"
    
    libstr= ' \\\n\t\t'.join(libs)
    docker_cmd= \
f'''
FROM python:3.7-slim-buster

RUN apt-get update \\
      && apt-get install -y \\
      # build-essential \\
      curl zip unzip

RUN pip3 install {libstr}

RUN curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip" \\
    &&   unzip -q awscliv2.zip \\
    &&   ./aws/install
    
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

'''
    # compile docker
    f= open(docker_fname, 'w')
    f.writelines(docker_cmd)
    f.close()
    
    # upload to AWS ECR
    assert ecr_base is not None, 'Need ECR path'
    
    cmd= 'aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin {}'.format(ecr_base)
    run_cmd(cmd, check= True)
    
    cmd= "aws ecr create-repository --repository-name {}".format(img_name) 
    run_cmd(cmd, check= False, dryrun= dryrun)
    cmd= "docker build -t {} {}".format(img_name, dirname)
    run_cmd(cmd, dryrun= dryrun)
    cmd= "docker tag {}:{} {}/{}:{}".format(img_name, img_tag, ecr_base, img_name, img_tag) # " base-layer:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/base-layer:latest
    run_cmd(cmd, dryrun= dryrun)
    cmd= "docker push {}/{}:{}".format(ecr_base, img_name, img_tag) # 667175760002.dkr.ecr.eu-central-1.amazonaws.com/base-layer:latest
    run_cmd(cmd, dryrun= dryrun)
    
    # clean up here unless
    if cleanup and (dirname is None):
        tmpdirname.cleanup()
    return docker_cmd
    


In [32]:
print(make_Docker(dirname= 'docker_test2/', img_name= 'sagemaker-processing-container-test2', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-processing-container-test2

** Non-zero status returned! **
docker build -t sagemaker-processing-container-test2 docker_test2/
Sending build context to Docker daemon  2.048kB
Step 1/6 : FROM python:3.7-slim-buster
 ---> a71a39002ed1
Step 2/6 : RUN apt-get update       && apt-get install -y       curl zip unzip
 ---> Using cache
 ---> 5329933107cf
Step 3/6 : RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1
 ---> Using cache
 ---> 7cc1e5a6c9f6
Step 4/6 : RUN curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"     &&   unzip -q awscliv2.zip     &&   ./aws/install
 ---> Using cache
 ---> 2c7e79b46881
Step 5/6 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 7a067ebdb646
Step 6/6 : ENTRYPOINT ["python3"]
 ---> Using cache
 --->

In [15]:
print(make_Docker(dirname= 'docker_test6/', img_name= 'another-sagemaker-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False, dryrun= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name another-sagemaker-test

** Non-zero status returned! **
docker build -t another-sagemaker-test docker_test6/
Sending build context to Docker daemon  2.048kB
Step 1/6 : FROM python:3.7-slim-buster
 ---> a71a39002ed1
Step 2/6 : RUN apt-get update       && apt-get install -y       curl zip unzip
 ---> Using cache
 ---> 5329933107cf
Step 3/6 : RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 		boto3
 ---> Using cache
 ---> d40ded52adab
Step 4/6 : RUN curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"     &&   unzip -q awscliv2.zip     &&   ./aws/install
 ---> Running in 3c9698a90ea7
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


AssertionError: Command execution failed. Abort!

In [25]:
run_cmd("ls -ltr", dryrun= False, check= True)

ls -ltr
total 9252
drwx------  2 root     root       16384 Aug  6  2021 lost+found
-rw-rw-r--  1 ec2-user ec2-user 5383698 Aug 26  2021 bank_clean.csv
-rw-rw-r--  1 ec2-user ec2-user 3544984 Aug 26  2021 train.csv
-rw-rw-r--  1 ec2-user ec2-user   32622 Aug 26  2021 Untitled.ipynb
drwxrwxr-x 51 ec2-user ec2-user    4096 Sep  1  2021 amazon-sagemaker-examples
drwxrwxr-x  2 ec2-user ec2-user    4096 Sep 29  2021 docker_test3
drwxr-xr-x  7 ec2-user ec2-user    6144 Sep 29  2021 efs
drwxrwxr-x  2 ec2-user ec2-user    4096 Sep 29  2021 docker_test1
drwxrwxr-x  2 ec2-user ec2-user    4096 Sep 29  2021 docker_test5
drwxrwxr-x  2 ec2-user ec2-user    4096 Sep 30  2021 docker_test4
-rw-rw-r--  1 ec2-user ec2-user   35316 Oct  1  2021 AWS processing test.ipynb
drwxrwxr-x  3 ec2-user ec2-user    4096 Oct  1  2021 s3_cache
drwxrwxr-x  2 ec2-user ec2-user   36864 Oct  1  2021 s3_cache2
drwxrwxr-x  2 ec2-user ec2-user   36864 Oct  1  2021 s3_cache3
drwxrwxr-x  2 ec2-user ec2-user    4096 Oct  1  202

In [18]:
print(make_Docker(dirname= 'docker_test5/', img_name= 'another-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-another-test
docker build -t sagemaker-another-test docker_test5/
docker tag sagemaker-another-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-another-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-another-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [5]:
print(make_Docker(dirname= 'docker_test1/', img_name= 'sagemaker-processing-container-test3', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-processing-container-test3
docker build -t sagemaker-processing-container-test3 docker_test1/
docker tag sagemaker-processing-container-test3:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test3:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test3:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [6]:
# failed
print(make_Docker(dirname= 'docker_test1/', img_name= 'blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name blah-blah-test
docker build -t blah-blah-test docker_test1/
docker tag blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [8]:
# success
print(make_Docker(dirname= 'docker_test1/', img_name= 'sagemaker-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-blah-blah-test
docker build -t sagemaker-blah-blah-test docker_test1/
docker tag sagemaker-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [10]:
# failed
print(make_Docker(dirname= 'docker_test1/', img_name= 'a-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name a-blah-blah-test
docker build -t a-blah-blah-test docker_test1/
docker tag a-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/a-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/a-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [14]:
print(make_Docker(dirname= 'docker_test1/', img_name= 'sagemaker-a-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-a-blah-blah-test
docker build -t sagemaker-a-blah-blah-test docker_test1/
docker tag sagemaker-a-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-a-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-a-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [13]:
print(make_Docker(dirname= 'docker_test1/', img_name= 'i-have-sagemaker-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name i-have-sagemaker-blah-blah-test
docker build -t i-have-sagemaker-blah-blah-test docker_test1/
docker tag i-have-sagemaker-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/i-have-sagemaker-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/i-have-sagemaker-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [12]:
# failed
print(make_Docker(dirname= 'docker_test1/', img_name= 'not-sage-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name not-sage-blah-blah-test
docker build -t not-sage-blah-blah-test docker_test1/
docker tag not-sage-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sage-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sage-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [11]:
#  fail
print(make_Docker(dirname= 'docker_test1/', img_name= 'not-sag-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name not-sag-blah-blah-test
docker build -t not-sag-blah-blah-test docker_test1/
docker tag not-sag-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sag-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sag-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [9]:
# success
print(make_Docker(dirname= 'docker_test1/', img_name= 'not-sagemaker-blah-blah-test', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name not-sagemaker-blah-blah-test
docker build -t not-sagemaker-blah-blah-test docker_test1/
docker tag not-sagemaker-blah-blah-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sagemaker-blah-blah-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/not-sagemaker-blah-blah-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [7]:
# success
print(make_Docker(dirname= 'docker_test1/', img_name= 'sagemaker-processing-container-test4', ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-processing-container-test4
docker build -t sagemaker-processing-container-test4 docker_test1/
docker tag sagemaker-processing-container-test4:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test4:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test4:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [6]:
print(make_Docker(ecr_base='667175760002.dkr.ecr.eu-central-1.amazonaws.com', cleanup= False))

aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name docker-auto-test
docker build -t docker-auto-test docker_test1
docker tag docker-auto-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/docker-auto-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/docker-auto-test:latest

FROM python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3 boto3==1.18.21 sagemaker==2.48.1 \
		boto3

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]




In [ ]:
docker build -t sagemaker-processing-container-test2 docker_test
aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name sagemaker-processing-container-test2
docker tag sagemaker-processing-container-test2:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test2:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-processing-container-test2:latest
    
    ECR base= 667175760002.dkr.ecr.eu-central-1.amazonaws.com
aws ecr create-repository --repository-name docker-auto-test
docker build -t docker-auto-test docker_test1
docker tag docker-auto-test:latest 667175760002.dkr.ecr.eu-central-1.amazonaws.com/docker-auto-test:latest
docker push 667175760002.dkr.ecr.eu-central-1.amazonaws.com/docker-auto-test:latest